In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
train = pd.read_csv('./data/train.csv',index_col='index')
train.head()

,event_id,X,Y,Z,TX,TY,chi2,signal,brick_number
index,,,,,,,,,
0,-999,66162.843750,65620.031250,0.0,0.166890,0.276674,2.988714,0.0,44
1,-999,43900.445312,67091.671875,0.0,-0.179967,-0.478795,1.901420,0.0,98
2,-999,37564.019531,26618.890625,55599.0,-0.267820,-0.282087,2.510245,0.0,64
3,-999,39380.027344,48135.425781,19395.0,0.265197,-0.577493,1.215219,0.0,63
4,-999,65040.781250,44018.468750,12930.0,-0.711161,-0.245592,2.973661,0.0,2


In [8]:
# оценим количество кирпичей
train.brick_number.unique().shape

(100L,)

In [37]:
# predict position of the track on the next layer
def getPosition(track,dz):
    '''
    track - track record (X,Y,Z,TX,TY)
    '''
    pos = np.array([track.X, track.Y, track.TX, track.TY])
    pos[0] +=  dz*np.tan(track.TX)
    pos[1] +=  dz*np.tan(track.TY)  
    return pos

In [15]:
brick2 = train.query('brick_number == 2')

In [17]:
brick2.query('signal == 1').event_id.unique()

array([15847, 20217,  4299], dtype=int64)

Примерный план обработки:
    <ul>
    <li> Разобрать треки по кирпичам
    <li> Отсортировать по расстоянию z, т.е. по слоям. Подумать о введении дополнительной переменной "слой"
    <li> Отсортировать по координате X
    <li> Начать перебор с начала кирпича.
    </ul>

In [89]:
idx1 = 3901599
idx2 = 3670692
prediction = getPosition(train.iloc[idx1,:],18102.0-16809.0)

In [133]:
dx = 20 # estimation of error along OX axis
dy = 20 # estimation of error along OY axis
dtx = 0.01
dty = 0.01

In [43]:
# add new feature - layer number

In [66]:
zPos = np.sort(event15847.Z.unique())
zPosDict = dict(enumerate(z)) # dictionary of positions (layerNumber => position)
zDiffDict = dict(enumerate(np.diff(zPos))) # dictionaru of distances (layerNumber => distance to the next layer)
layerDict = {v: k for k, v in zPosDict.iteritems() } # dictionary of layer number (position => layerNumber)

In [78]:
event15847 = event15847.assign(layer = event15847.Z.map(layerDict))

In [87]:
event15847.sort_values(by='layer',ascending=True).head()

,event_id,X,Y,Z,TX,TY,chi2,signal,brick_number,layer
index,,,,,,,,,,
3901599,15847,35281.085938,28812.714844,16809.0,0.096675,0.051116,0.568254,1.0,2,0
3670692,15847,35407.148438,28880.599609,18102.0,0.101562,0.052474,0.318941,1.0,2,1
101836,15847,35535.875000,28950.218750,19395.0,0.104953,0.056952,0.966164,1.0,2,2
2639629,15847,35667.257812,29018.583984,20688.0,0.101516,0.056934,1.357672,1.0,2,3
3443804,15847,35798.773438,29085.591797,21981.0,0.102604,0.048461,1.086284,1.0,2,4


In [100]:
brick = brick2

In [107]:
# create dictionaries
zPos = np.sort(brick.Z.unique())
zPosDict = dict(enumerate(zPos)) # dictionary of positions (layerNumber => position)
zDiffDict = dict(enumerate(np.diff(zPos))) # dictionaru of distances (layerNumber => distance to the next layer)
layerDict = {v: k for k, v in zPosDict.iteritems() } # dictionary of layer number (position => layerNumber)

# add extra feature - layer number
brick = brick.assign(layerId = brick.Z.map(layerDict))

In [125]:
idx1 = 3901599
idx2 = 3670692
layerId = brick.loc[idx1].layerId
prediction = getPosition(brick.loc[idx1],zDiffDict.get(layerId))
prediction

array([  3.54064776e+04,   2.88788655e+04,   9.66750383e-02,
         5.11160716e-02])

In [157]:
brick.loc[idx1]

event_id        15847.000000
X               35281.085938
Y               28812.714844
Z               16809.000000
TX                  0.096675
TY                  0.051116
chi2                0.568254
signal              1.000000
brick_number        2.000000
layerId            13.000000
Name: 3901599, dtype: float64

In [132]:
brick.loc[idx2]

event_id        15847.000000
X               35407.148438
Y               28880.599609
Z               18102.000000
TX                  0.101562
TY                  0.052474
chi2                0.318941
signal              1.000000
brick_number        2.000000
layerId            14.000000
Name: 3670692, dtype: float64

In [139]:
set1 = brick.query('layerId == @layerId+1 & X > @prediction[0]-@dx & X < @prediction[0]+@dx & Y > @prediction[1]-@dy & Y < @prediction[1]+@dy & TX > @prediction[2]-@dtx & TX < @prediction[2]+@dtx & TY > @prediction[3]-@dty & TY < @prediction[2]+@dty')
set1

,event_id,X,Y,Z,TX,TY,chi2,signal,brick_number,layerId
index,,,,,,,,,,
3670692,15847,35407.148438,28880.599609,18102.0,0.101562,0.052474,0.318941,1.0,2,14


In [154]:
layerId = 13
layer0 = brick.query('layerId == @layerId')
layer1 = brick.query('layerId == @layerId+1')
#prediction = getPosition(brick.loc[idx1],zDiffDict.get(layerId))

In [169]:
dz = zDiffDict.get(layerId)
for trackId, trackRec in layer0.iterrows():
    predPosition = getPosition(trackRec,dz)
    #print trackId
    #layer1.query('X > @predPosition[0]-@dx & X < @predPosition[0]+@dx & Y > @predPosition[1]-@dy & Y < @predPosition[1]+@dy & TX > @predPosition[2]-@dtx & TX < @predPosition[2]+@dtx & TY > @predPosition[3]-@dty & TY < @predPosition[3]+@dty')
    q = layer1.query('X > @predPosition[0]-@dx & X < @predPosition[0]+@dx & Y > @predPosition[1]-@dy & Y < @predPosition[1]+@dy')
    if q.shape[0] >0:
        pri

Empty DataFrame
Columns: [event_id, X, Y, Z, TX, TY, chi2, signal, brick_number, layerId]
Index: []
Empty DataFrame
Columns: [event_id, X, Y, Z, TX, TY, chi2, signal, brick_number, layerId]
Index: []
Empty DataFrame
Columns: [event_id, X, Y, Z, TX, TY, chi2, signal, brick_number, layerId]
Index: []
Empty DataFrame
Columns: [event_id, X, Y, Z, TX, TY, chi2, signal, brick_number, layerId]
Index: []
Empty DataFrame
Columns: [event_id, X, Y, Z, TX, TY, chi2, signal, brick_number, layerId]
Index: []
Empty DataFrame
Columns: [event_id, X, Y, Z, TX, TY, chi2, signal, brick_number, layerId]
Index: []
Empty DataFrame
Columns: [event_id, X, Y, Z, TX, TY, chi2, signal, brick_number, layerId]
Index: []
Empty DataFrame
Columns: [event_id, X, Y, Z, TX, TY, chi2, signal, brick_number, layerId]
Index: []
Empty DataFrame
Columns: [event_id, X, Y, Z, TX, TY, chi2, signal, brick_number, layerId]
Index: []
Empty DataFrame
Columns: [event_id, X, Y, Z, TX, TY, chi2, signal, brick_number, layerId]
Index: []


KeyboardInterrupt: 